# Unleash the Ray - Grid Search

In [1]:
%load_ext autoreload
%autoreload 2

from dependencies import *

Loading dependencies we have already seen...
Done...


In [2]:
import ray
from ray import tune

### Let's start Ray

In [3]:
ray.init(num_cpus=6, num_gpus=0, include_webui=True)

2020-06-11 11:37:26,802	INFO resource_spec.py:204 -- Starting Ray with 35.3 GiB memory available for workers and up to 17.65 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-11 11:37:27,097	INFO services.py:1168 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.39',
 'raylet_ip_address': '192.168.1.39',
 'redis_address': '192.168.1.39:18030',
 'object_store_address': '/tmp/ray/session_2020-06-11_11-37-26_771023_38101/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-06-11_11-37-26_771023_38101/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-06-11_11-37-26_771023_38101'}

After initialisation the [Ray Dashboard](https://docs.ray.io/en/master/ray-dashboard.html) is available on the **webui_url** port

In [4]:
ray.shutdown()

## Setup some raytune compatible training code

In [ ]:

# differences from what we've seen before, this is an end to end training function
# where we are loading the dataset running our complete train and test loop whilst
# 
def e2e_simple_training(config):
    
    #threadsafe
    X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)
    
    # chose your CV strategy
    splitter = StratifiedKFold(n_splits=5)
    
    # run k fold training and testing
    f1_scores = [] # keep hold of all individual scores
    for train_ind, test_ind in splitter.split(X, y):
        pipeline = make_pipeline(RobustScaler(),
                                  RandomForestClassifier(random_state=42))

        pipeline.set_params(**config)
        pipeline.fit(X[train_ind], y[train_ind])
        
        y_pred = pipeline.predict(X[test_ind])
        
        f1_scores.append(f1_score(y_pred, y[test_ind]))
    
    # use tunes reporter
    tune.track.log(mean_f1_score=np.array(f1_scores).mean(),
                std_f1_score=np.array(f1_scores).std(),
                # and we can actually add any metrics we like
                done=True)

### TODO convert this to a set of ray search spaces

The Ray config object is freeform, we imprint our own structure.

However, tunable parameters need to be represented by tune distribution object >> [read the docs](https://docs.ray.io/en/latest/tune/api_docs/grid_random.html?highlight=tune.grid#random-distributions-api)

In [ ]:
ray_tuning_config = {
    'randomforestclassifier__n_estimators': tune.grid_search([1,5,15,50,100])
}

In [ ]:
analysis = tune.run(
                e2e_simple_training,
                config=ray_tuning_config,
                resources_per_trial=dict(cpu=1, gpu=0),
                local_dir="~/ray_results/grid_search")

In [ ]:
df = analysis.dataframe()
print(df.columns)
df.head()

In [ ]:
print("Best config: ", analysis.get_best_config(metric="mean_f1_score"))

In [ ]:
from scipy.stats import norm

def plot_some_tune_results(df):
    fig, ax = plt.subplots(1, 1, figsize=(16,6))
    x = np.linspace(0.85, 1.0, 100)

    n_estimators = df['config/randomforestclassifier__n_estimators'].values.tolist()

    lines = []
    for mu, sigma in zip(df['mean_f1_score'], df['std_f1_score']):
        pdf = norm.pdf(x, mu, sigma)
        line, = ax.plot(x, pdf, alpha=0.6)
        ax.axvline(mu, color=line.get_color())
        ax.text(mu, pdf.max(), f"{mu:.3f}", color=line.get_color(), fontsize=14)
        lines.append(line)

    plt.legend(handles=lines, labels=n_estimators, title="n estimators")
    ax.set_title(f"Average F1 Scores")
    
plot_some_tune_results(df)

## Really increase the size of the search space

In [ ]:
#
# 6D search space - 960 combinations - 4800 calls to fit
#

ray_tuning_config = {
    'randomforestclassifier__n_estimators': tune.grid_search([1,5,15,50,100]),
    'randomforestclassifier__criterion': tune.grid_search(['gini', 'entropy']),
    'randomforestclassifier__max_features': tune.grid_search(['auto', 'sqrt', 'log2']),
#     'randomforestclassifier__bootstrap': tune.grid_search([True, False]),
#     'randomforestclassifier__min_samples_leaf': tune.grid_search([1,2,3,4]),
#     'randomforestclassifier__min_samples_split': tune.grid_search([3,4,5,6])
}

In [ ]:
analysis = tune.run(
                e2e_simple_training,
                config=ray_tuning_config,
                resources_per_trial=dict(cpu=1, gpu=0))

In [ ]:
print("Best config: ", analysis.get_best_config(metric="mean_f1_score"))

In [ ]:
df = analysis.dataframe()
top_n_df = df.nlargest(10, "mean_f1_score")

In [ ]:
plot_some_tune_results(top_n_df)

In [ ]:
%load_ext tensorboard.notebook

In [ ]:
from tensorboard import notebook
notebook.display(height=1000) 
%tensorboard --logdir "~/ray_results/grid_search"

### Once you are all done, shutdown Ray

In [ ]:
ray.shutdown()

exit

So what have we just done?
 - shown how the performance of a model with a single set of hyperparameters affects our score
 - shown that kfold cv can give us a picture of that distribution which is easier to compute that out-of-bootstrap scores and has less leakage
 - so which model to choose? == in this case which data split, which is not what we want, we want to chose a robust estimator so we want to find a model with the best score distribution for a given set of hyperparameters
 - (note: if we want to do hyperparaemter tuning and want our model selection to be unbiaesed weshould do nested Cross Validaiton, but for simpliciy of the code we are sticking with kfold. See: xxxxx_xxxx_xx.ipynb for the nested cross validaiton example)